# 네이버 매장정보 크롤링

#### 자료 
https://www.localdata.go.kr/devcenter/dataDown.do?menuNo=20001

1. 관광식당, 일반음식점, 휴게음식점 데이터셋 다운로드(07_24_01_P_.csv.zip, 07_24_04_P_.csv.zip , 07_24_05_P_.csv.zip)
2. 다운로드한 파일들을 data 폴더에 넣어주세요. (압축풀지말고)

문제 
1. 강남구 매장이지만, 네이버로 검색하는 경우 강남구가 아닌 다른 지역의 매장명이 검색됨
2. 데이터 반영이 빠른편이지만, 네이버에 매장이 없는 경우가 있음 (없는 매장 : 맥도날드, 교촌, 굽네) 
3. 검색하면 전혀 다른 업체가 나오는 경우가 있음 -> 네이버 업태구분명을 크롤링해와서 참고하여 반영해야 
4. 나라에서 제공하는 data를 사용하는 경우 인기도를 알기 어려움. 아닌가. -> 네이버 지도 api를 사용해서 점수를 부여할까? 
5. 자료 업데이트 시기 문제 - data를 다운로드 받아서 사용하는 경우, 매장 정보는 계속 바뀐는데 업데이트가 늦잖아... 매장 정보의 변동이 생기 사람이 계속 업로드를 해야 하잖아.. 

In [ ]:
# 웹 드라이버 설정
from selenium import webdriver  
from webdriver_manager.chrome import ChromeDriverManager 

# 대기 관련 라이브러리
from selenium.webdriver.support.ui import WebDriverWait 
from selenium.webdriver.support import expected_conditions as EC 

# 예외 처리 관련 라이브러리
from selenium.common.exceptions import TimeoutException, NoSuchElementException  

# 웹 요소 찾기 관련 라이브러리
from selenium.webdriver.common.by import By  
from selenium.webdriver.support.ui import Select  
from selenium.webdriver.common.keys import Keys  

# 그 외 
import time 
import warnings
warnings.filterwarnings('ignore')
from bs4 import BeautifulSoup 
import numpy as np  
import pandas as pd 
import re  
from tqdm import tqdm  # 반복문 진행 상황 시각화 모듈
import zipfile
import os

In [ ]:
# 지역 설정
city = '서울특별시'
gu = '강남구'

In [ ]:
# zip 파일 압축 풀기 

folder_path = './data'
zip_folder = './data/zip_data'

# data 폴더 안에 있는 압축 파일들 압축 풀기
for file in os.listdir(folder_path):
    zip_file_path = os.path.join(folder_path, file)
    if zipfile.is_zipfile(zip_file_path): 
        with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
            zip_ref.extractall(zip_folder)

# csv 파일이 큰 순서대로 정렬
csv_files_sorted = sorted([os.path.join(zip_folder, f) for f in os.listdir(zip_folder) if f.endswith('.csv')],
                          key=lambda x: os.path.getsize(x),reverse=True)
csv_files_sorted

In [ ]:
# 첫번째 데이터 불러오기 (데이터 크기가 커서 둘로 나눈뒤 합침)
chunk_size = 1000000
chunks = pd.read_csv(csv_files_sorted[0], encoding='cp949', encoding_errors='ignore', chunksize=chunk_size)
df1_1 = next(chunks)  # 첫 번째 청크(1,000,000 행) 읽기
df1_2 = pd.concat(chunks, ignore_index=True)  # 나머지 데이터 읽기

# 현재 운영하고 있는 매장만 가져와서 합치기 
df1_1.drop(df1_1[df1_1['영업상태명'] == '폐업'].index, inplace=True)
df1_2.drop(df1_2[df1_2['영업상태명'] == '폐업'].index, inplace=True)
df1 = pd.concat([df1_1, df1_2], ignore_index=True)


In [ ]:
df1.info()

In [ ]:
df1.columns

In [ ]:
# 두번째 데이터 불러오기 
df2 = pd.read_csv(csv_files_sorted[1], encoding='cp949', encoding_errors='ignore')
df2.columns

In [ ]:
# 세번째 데이터 불러오기 
df3 = pd.read_csv(csv_files_sorted[2], encoding='cp949', encoding_errors='ignore')
df3.columns

In [ ]:
# 필요한 컬럼만 들고 오기 
selected_columns = ['영업상태명', '소재지전체주소', '도로명전체주소', '사업장명', '최종수정시점','데이터갱신일자','업태구분명', '좌표정보(x)', '좌표정보(y)','위생업태명']

df1 = df1[selected_columns]

# 지역구 구분하기 
df1['시도'] = df1['소재지전체주소'].apply(lambda x: x.split(' ', 1)[0] if pd.notna(x) else None)
df1['지역구'] = df1['소재지전체주소'].apply(lambda x: x.split(' ')[1] if pd.notna(x) and len(x.split(' ')) > 1 else None)



# 현재 운영하고 있는 매장만 가져오기
df2.drop(df2[df2['영업상태명'] == '폐업'].index, inplace=True)

# 필요한 컬럼만 들고 오기 
df2 = df2[selected_columns]

# 지역구 구분하기 
df2['시도'] = df2['소재지전체주소'].apply(lambda x: x.split(' ', 1)[0] if pd.notna(x) else None)
df2['지역구'] = df2['소재지전체주소'].apply(lambda x: x.split(' ')[1] if pd.notna(x) and len(x.split(' ')) > 1 else None)


# 현재 운영하고 있는 매장만 가져오기
df3.drop(df3[df3['영업상태명'] == '폐업'].index, inplace=True)

# 필요한 컬럼만 들고 오기 
df3 = df1[selected_columns]

# 지역구 구분하기 
df3['시도'] = df3['소재지전체주소'].apply(lambda x: x.split(' ', 1)[0] if pd.notna(x) else None)
df3['지역구'] = df3['소재지전체주소'].apply(lambda x: x.split(' ')[1] if pd.notna(x) and len(x.split(' ')) > 1 else None)

In [ ]:
# 데이터 합치고 중복제거
origin_df = pd.concat([df1, df2, df3], ignore_index=True).drop_duplicates()
origin_df.head(10)

In [ ]:
origin_df.info()

In [ ]:
# # 폴더 안에 있는 파일을 반복해서 df에 저장하고 싶었는데... 메모리 이슈 실패 

# zip_folder = './data/zip_data'
# chunk_size = 1000000
# df_list = []
# n = 1


# for file in os.listdir(zip_folder):
#     file_path = os.path.join(zip_folder, file)
    
#     # 파일 크기가 800MB 이상인지 확인
#     if os.path.getsize(file_path) >= 800000000:
#         chunks = pd.read_csv(file_path, encoding='cp949', encoding_errors='ignore', chunksize=chunk_size)
#         df1_1 = next(chunks)  # 첫 번째 청크(1,000,000 행) 읽기
#         df1_2 = pd.concat(chunks, ignore_index=True)  # 나머지 데이터 읽기

#         # 현재 운영하고 있는 매장만 가져오기
#         df1_1.drop(df1_1[df1_1['영업상태명'] == '폐업'].index, inplace=True)
#         df1_2.drop(df1_2[df1_2['영업상태명'] == '폐업'].index, inplace=True)
#         df_0 = pd.concat([df1_1, df1_2], ignore_index=True)
#         df_list.append('df_0')

#     else:
#         for n in range(1, 4):
#             df_name = 'df_{}'.format(n)
#             globals()[df_name] = pd.read_csv(file_path, encoding='cp949', encoding_errors='ignore')
#             df_list.append(globals()[df_name])
                
# df_list

In [ ]:
# '시도' 컬럼이 '서울특별시'면서 '지역구' 값이 '강남구'인 행만 필터링
df = origin_df[(origin_df['시도'] == city ) & (origin_df['지역구'] == gu )]
df.head()

# csv 파일로 저장
df.to_csv('filtered_data_{}{}.csv'.format(city,gu), index=False)


In [ ]:
df.head(5)

In [ ]:
df.isna().sum()

In [ ]:
df.info()

In [ ]:
# 업태구분명 확인
unique_values = df['업태구분명'].unique()
unique_values

In [ ]:
# 카페, 술, 출장요리 류 삭제
제외 = ['전통찻집','호프/통닭','뷔페식','출장조리','정종/대포집/소주방','이동조리', '감성주점','까페','라이브카페','키즈카페','커피숍','편의점', '일반조리판매','아이스크림', '떡카페', '철도역구내', '푸드트럭', '과자점', '다방', '관광호텔']
for i in 제외 :
    df = df.drop(df[df['업태구분명'] == i].index)
df

In [ ]:
# Test할 매장
df_top10 = df.head(10)
df_top10

# 셀레니움

In [89]:
# webdriver_manager를 사용하여 ChromeDriver 다운로드 및 설정
driver = webdriver.Chrome(ChromeDriverManager().install())
# 주소 이동
url = 'https://map.naver.com/'
driver.get(url)
time.sleep(1)


In [84]:
# 지역+메뉴 이름 조합으로 url에 미리 포함시키는 driver.get 함수 , count는 검색했을때 조회하는 범위
def naver_finder(place,count) :

  # 검색어를 네이버 url에 포함시켜 이동 + 검색 
  driver.get('https://map.naver.com/p/search/{}'.format(place))

  # 저장 
  naver_df = pd.DataFrame()
  time.sleep(3)  
  try : 
    # 목록의 1~9번째까지 반복
    for num in range(1,count+1) :
      # frame을 3갤 나눔 searchIframe(왼쪽), entryIframe(오른쪽),default_content(기본)

      # 왼쪽 프레임
      searchIframe = driver.find_element(By.ID,'searchIframe')
      driver.switch_to.frame(searchIframe)

      try : 
        # {}안에 num 입력 (1번째, 2번째...)
        # 각 카드 상단을 클릭하여 WebDriverWait을 활용 카드 상단 XPATH가 보일때까지 3초 대기 
        driver.find_element(By.XPATH,'//*[@id="_pcmap_list_scroll_container"]/ul/li[{}]/div[1]/a'.format(num)).click()

      except :
        # 가능하지 않다면 스크롤 내리고 최대 3초까지 기다렸다가 클릭
        # tag 내용이 안 보이면 해당 tag에서 스크롤 내림
        # click()이 안되는 경우 클릭을 대체하여 사용하기 
        driver.find_element(By.TAG_NAME,'body').send_keys(Keys.PAGE_DOWN)
        card_clik = driver.find_element(By.XPATH,'//*[@id="_pcmap_list_scroll_container"]/ul/li[{}]/div[1]/a'.format(num))
        driver.execute_script('arguments[0].click();',card_clik)

      # 기본 프레임
      time.sleep(1)
      driver.switch_to.default_content()

      # 오른쪽 프레임
      entryIframe = driver.find_element(By.ID,'entryIframe')
      driver.switch_to.frame(entryIframe)

      # 가게 이름, 변수로 지정
      restaurant_name = driver.find_element(By.CLASS_NAME,'Fc1rA').text

      # 가게 별점이 있는 경우에만 try 
      try :
        review_star = driver.find_element(By.CLASS_NAME,'PXMot').text
        review_star = re.sub('방문자리뷰',"",review_star)
      except :
        review_star = 0
        pass 

      # 기본 설정으로 돌아오기
      time.sleep(1)
      driver.switch_to.default_content()

      print(restaurant_name,review_star)
      
      # 수정 필요 
      df = pd.DataFrame({'가게명':[restaurant_name],'네이버별점' : [review_star]})
      naver_df = pd.concat([naver_df,df])
      naver_df.drop_duplicates() # 중복 제거 
  except :
    print('정상적으로 작동이 되지 않습니다.')

    return naver_df


In [92]:
for i in df_top10['사업장명']:
    df_sample = naver_finder(i,2)
    time.sleep(5)

정상적으로 작동이 되지 않습니다.
정상적으로 작동이 되지 않습니다.
정상적으로 작동이 되지 않습니다.
정상적으로 작동이 되지 않습니다.
정상적으로 작동이 되지 않습니다.
하늘사다리 별점
4.43
은희네 온집닭떡볶이 상도본점 별점
4.47
정상적으로 작동이 되지 않습니다.
정상적으로 작동이 되지 않습니다.
정상적으로 작동이 되지 않습니다.
정상적으로 작동이 되지 않습니다.


""


""
